In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
dataset = pd.read_csv('/kaggle/input/vehicle-dataset-from-cardekho/car data.csv')
dataset.head()

In [ ]:
dataset.shape

In [ ]:
dataset.isnull().sum()

In [ ]:
print(dataset['Seller_Type'].value_counts())
print(dataset['Transmission'].value_counts())
print(dataset['Owner'].value_counts())
print(dataset['Fuel_Type'].value_counts())

In [ ]:
dataset.describe()

In [ ]:
final_df = dataset.drop('Car_Name', axis=1)
final_df.head()

In [ ]:
final_df['Years_old'] = 2020 - final_df['Year']
final_df.head()

In [ ]:
final_df.drop('Year', axis=1, inplace=True)
final_df.head()

In [ ]:
final_df = final_df[final_df['Fuel_Type']!='CNG']
final_df = final_df[final_df['Owner']!=3]

In [ ]:
final_df = pd.get_dummies(final_df, drop_first=True)
final_df.head()

In [ ]:
sns.pairplot(final_df)

In [ ]:
sns.heatmap(final_df.corr(), annot=True)

In [ ]:
X = final_df.iloc[:, 1:]
y = final_df.iloc[:, 0]

In [ ]:
X.head()

In [ ]:
y.head()

In [ ]:
from sklearn.ensemble import ExtraTreesRegressor

model = ExtraTreesRegressor()
model.fit(X,y)

In [ ]:
pd.Series(model.feature_importances_, index=X.columns).sort_values(ascending=False).nlargest(5).plot(kind='barh')

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train.shape

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf_random = RandomForestRegressor()

In [ ]:
n_estimators = [int(x) for x in np.linspace(100, 1200, 12)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(5,30, 6)]
min_samples_split = [2,5,10,15,100]
min_samples_leaf = [1,2,5,10]

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
random_grid = {'n_estimators':n_estimators ,
'max_features': max_features,
'max_depth':max_depth ,
'min_samples_split' :min_samples_split ,
'min_samples_leaf' :min_samples_leaf }

print(random_grid)

In [ ]:
rf = RandomizedSearchCV(rf_random, param_distributions=random_grid, scoring='neg_mean_squared_error', n_iter=10, cv=5, n_jobs=-1, verbose=2, random_state=42)

In [ ]:
rf.fit(X_train, y_train)

In [ ]:
y_pred = rf.predict(X_test)
y_pred

In [ ]:
sns.kdeplot(y_test-y_pred)

In [ ]:
plt.scatter(y_test, y_pred)

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

print(mean_squared_error(y_test, y_pred, squared=False))
print(r2_score(y_test, y_pred))